In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 그냥 확인절차

In [2]:
df = pd.read_csv("blind_50s.csv", delimiter=",")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18554 entries, 0 to 18553
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  18554 non-null  int64 
 1   contents    18554 non-null  object
 2   time        18554 non-null  object
 3   keyword     18554 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.9+ KB


In [4]:
df.head()

,Unnamed: 0,contents,time,keyword
0,0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 ...,작성시간\n2020.07.08.,디지털 드로잉
2,2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터...,작성시간\n04.12,디지털 드로잉
4,4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&...,작성시간\n2019.06.12.,디지털 드로잉


In [5]:
list(df["contents"][:5])

['라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...',
 '아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 형 누나들??',
 '손으로 그리는 강의 추천하는 거 있나요?',
 '그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요~?',
 '요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&합정&DMC 쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요? 한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요ㅠㅠ 이번에 D뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요!! 취미로 하시는 분들 계시면 댓글 부탁드려요!^^']

In [6]:
#df2 = df[::3]  #3번째 지나면 1개씩 뽑기 (0, 3, 6...)
#df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6185 entries, 0 to 18552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6185 non-null   int64 
 1   contents    6185 non-null   object
 2   time        6185 non-null   object
 3   keyword     6185 non-null   object
dtypes: int64(1), object(3)
memory usage: 193.4+ KB


In [7]:
df2.head()   

,Unnamed: 0,contents,time,keyword
0,0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
3,3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터...,작성시간\n04.12,디지털 드로잉
6,6,갤럭시탭 최신형으로 있어서 다른건 다필요없고 디지털 드로잉좀 배워보고싶어서 아이패드...,작성시간\n10.22,디지털 드로잉
9,9,보통 아이패드 많이 쓰더라구? 나는 갤탭뿐이고... 갤탭으로 한번 해보고싶은데 어디...,작성시간\n10.12,디지털 드로잉
12,12,재밌닷,작성시간\n07.31,드로잉


# 데이터 전처리 (url,숫자,영어,#,특수문자제거, 정규화 )

In [8]:
df = pd.read_csv("blind_50s.csv", delimiter=",")

In [9]:
data = df.drop(columns=['Unnamed: 0'], axis=1)

In [10]:
data.head()

,contents,time,keyword
0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 ...,작성시간\n2020.07.08.,디지털 드로잉
2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터...,작성시간\n04.12,디지털 드로잉
4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&...,작성시간\n2019.06.12.,디지털 드로잉


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18554 entries, 0 to 18553
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   contents  18554 non-null  object
 1   time      18554 non-null  object
 2   keyword   18554 non-null  object
dtypes: object(3)
memory usage: 435.0+ KB


In [12]:
#conents에 해당하는 부분만 'docs'라는 리스트로 만듦

docs = list(data['contents'])

In [ ]:
docs

In [14]:
import re
import requests
from bs4 import BeautifulSoup


from konlpy.tag import Okt 
okt = Okt()

In [15]:
# http로 시작되는 url제거
remove_url = [re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", str(content)) for content in docs]

# 숫자 제거
remove_num = [re.sub(r'\d+',' ', str(content)) for content in remove_url]

# 영어 제거 
remove_alpha = [re.sub('[a-zA-Z]' , ' ', str(content)) for content in remove_num]

# Hashtag 제거
remove_tag = [re.sub('[#]+[0-9a-zA-Z_]+', ' ', str(content)) for content in remove_alpha]

# 특수문자 제거
remove_special = [re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣]', ' ', str(content)) for content in remove_tag]

#정규식(Regular Expression)이용해서 정규화
lawdata = [re.sub(r"[^가-힣A-Za-z0-9]", " ", str(content)) for content in remove_special]


   # 쓰레기 단어 제거
   #text = re.sub('[&]+[a-z]+', ' ', text)

   # 특정 문자 제거
   #for i in ['!','@','#','*'] :
       #new_id = new_id.replace(i, "")

   # 띄어쓰기 제거
   #text = text.replace('\n',' ')

In [16]:
lawdata[:20]

['라고 하기엔  개 뿐이지만     배우기 시작한지는 얼마 안되서 아직은 기초단계   ',
 '아이패드    타블렛 뭐가 나을까   클래스   인가 그거 들을만 할까 들어본 형 누나들  ',
 '손으로 그리는 강의 추천하는 거 있나요 ',
 '그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요  ',
 '요즘 반복되는 업무에 치이다보니 취미 좀 정적인것  도전해보고 싶은데요 혹시 일산 합정     쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요  한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요   이번에  뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요   취미로 하시는 분들 계시면 댓글 부탁드려요   ',
 '왜 학창시절에보면 남자애들 그림    못그리잖아 글씨도 더럽고 그게난데 그림그리는걸 너무하고싶어 근데 도화지에 그리자신은없어 디지털드로잉 유튜브에 강의많던데 보고하면 많이늘까 ',
 '갤럭시탭 최신형으로 있어서 다른건 다필요없고 디지털 드로잉좀 배워보고싶어서 아이패드 살려고하거든 최신형 필요없고 아이팬슬 지원하는거 사면 좋을꺼같은데 어떤걸 사면 좋을까  아이패드  세대개 제일 무난할까 ',
 '손그림만 그리다가 아이패드 샀는데 어플 추천해줄수있어  프로크리에이트나 클립스튜디오 많이 쓰는것같은데 디지털 드로잉은 처음이고 포토샵 정도 다룰줄 알아           유튜브랑 인터넷 이것저것 찾아보는데 엄청 방대하네',
 '아이패드 세대    아이패드 프로  세대 중에 어떤게 나을까 디지털 드로잉 배워보려고 하는데 초보라    프로는 에바인가 프로는 가격때매    세대는 가격은 괜찮은데 기능 까는 리뷰가 많아서 뭐가 나을지 고민즁     ',
 '보통 아이패드 많이 쓰더라구  나는 갤탭뿐이고    갤탭으로 한번 해보고싶은데 어디서부터 시작해야할지 모르겠어 보통 강의같은걸로 시작하나      미대준비했었어서 그림은 잘 

# 형태소 분석 관련 코드(여기서는 okt.nouns 이용해서, 토크나이즈 후 명사만 골라냄)

In [ ]:
#morphs는 형태소 단위로 구문을 분석(형태소 단위로 토크나이즈)

lawdata_tokened_morphs = []

for doc in lawdata:
    newdoc = okt.morphs(doc)
    lawdata_tokened_morphs.append(newdoc)

In [ ]:
lawdata_tokened_morphs[:2]

In [ ]:
#명사만 추출해줌(토크나이즈해서 리스트 형태로 명사만 추출)

lawdata_tokened_nouns = []

for doc in lawdata:
    newdoc = okt.nouns(doc)
    lawdata_tokened_nouns.append(newdoc)

In [ ]:
lawdata_tokened_nouns[:2]

In [ ]:
#각 품사를 태깅하는 역할. 
#품사를 태깅한다는 것은 주어진 텍스트를 형태소 단위로 나누고, 나눠진 각 형태소를 그에 해당하는 품사와 함께 리스트화 하는 것을 의미한다. 
#형태소 분석으로, 형태소 단위 토크나이징
#pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True)]


pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True) 
                 if ('Noun' in p or 'Adjective' in p or 'Verb' in p or 'Adverb' in p) 
                 and '보다' not in p and '하다' not in p]
lawdata_tokened = [pos(doc) for doc in lawdata]

In [ ]:
lawdata_tokened[:2]

In [ ]:
print(lawdata_tokened[1])
lawdata_tokened[1][0].split('/')

In [ ]:
lawdata_tokened_filtered = []

for doc in lawdata_tokened:
    newdoc = [d for d in doc if d.split('/')[1] == 'Noun' or d.split('/')[1] == 'Adjective' or d.split('/')[1] == 'Adverb']
    lawdata_tokened_filtered.append(newdoc)

In [ ]:
lawdata_tokened_filtered[:2]

In [17]:
#명사만 추출하는 함수 정의(토크나이즈해서 명사만 추출)

import string
def noun_process(text):
    nouns = okt.nouns(text)
    #nouns = [word for word in nouns if len(word) > 1]
    return nouns

In [18]:
print(lawdata[0])
print(noun_process(lawdata[0]))

라고 하기엔  개 뿐이지만     배우기 시작한지는 얼마 안되서 아직은 기초단계   
['개', '뿐', '한지', '얼마', '은', '기초', '단계']


# CountVectorizer해서 word of bags 만듦 -> 문서전체를 doc-term matrix로 변환

In [19]:
#CountVectorizer: 토큰이 문서별로 몇 번 등장했는지 행렬로 정리해주는 모듈
#CountVectorizer해서 word of bags만들어주는 'bow_transformer' 함수 정의

bow_transformer = CountVectorizer(analyzer=noun_process).fit(lawdata)

In [20]:
#총 단어갯수

len(bow_transformer.vocabulary_)

20125

In [21]:
# 0번째줄 문장의 단어는 어느 위치의 벡터인가?, 몇 번 나오는가?

bow_0 = bow_transformer.transform([lawdata[0]])
print(bow_0)

  (0, 365)	1
  (0, 2152)	1
  (0, 3316)	1
  (0, 8089)	1
  (0, 11405)	1
  (0, 13063)	1
  (0, 19045)	1


In [22]:
# 이제 모든 lawdata를 변형

cv_lawdata= bow_transformer.transform(lawdata)

In [23]:
# CountVectorizer 적용한 lawdata 확인

print('Shape of Sparse Matrix: ', cv_lawdata.shape)
print('Amount of Non-Zero occurrences: ', cv_lawdata.nnz)

Shape of Sparse Matrix:  (18554, 20125)
Amount of Non-Zero occurrences:  267165


In [24]:
# 그런데 여기서 주의해야할 것 하나.
# docs는 현재 document-term matrix 로 구성됨
# NMF를 적용하기 위해서는 term-document matrix가 필요하다.
# 즉, 현재의 Matrix를 transposed matrix(전치행렬)로 바꿔줘야 함.

type(cv_lawdata)

scipy.sparse.csr.csr_matrix

In [25]:
# compressed sparse(희소행렬: 행렬의 값이 대부분 0일때) row matrix인 docs를 transpose 한다. (행열바꾸기)

cv_lawdata_trans = cv_lawdata.transpose()
type(cv_lawdata_trans) 

print('Shape of Sparse Matrix: ', cv_lawdata_trans.shape)
print('Amount of Non-Zero occurrences: ', cv_lawdata_trans.nnz)

Shape of Sparse Matrix:  (20125, 18554)
Amount of Non-Zero occurrences:  267165


# NMF 모델 객체 생성

In [27]:
# NMF 모델 객체 생성
# n_topics: 매칭해줄 분류객체(베이킹, 요리..)

from sklearn.decomposition import NMF

n_topics = 25
model = NMF(n_components=n_topics)

In [28]:
# NMF 모델 학습
W = model.fit_transform(cv_lawdata_trans)

In [29]:
H = model.components_

In [30]:
H.shape

(25, 18554)

In [34]:
# index 0번 문서의 클러스터링 결과 점수

print(H[:,0])

[0.00112982 0.         0.00473307 0.00561025 0.         0.00692328
 0.         0.00091081 0.00448554 0.0081309  0.0079448  0.00023681
 0.00379817 0.00016232 0.00085665 0.00913391 0.00705321 0.00240655
 0.         0.0033583  0.00361965 0.00617247 0.00021059 0.
 0.00646394]


In [35]:
# 전체 글 별로 best점수에 해당하는 클러스터 label결정

pred_labels = H.argmax(axis=0)

In [36]:
# label예측한 글 개수 = 원래 글 개수

len(pred_labels)

18554

In [58]:
#10번째글까지 예측한 각 글에 대한 label 번호

pred_labels[:10]

array([15, 23, 16,  1,  7, 21, 22,  7, 23, 19])

In [59]:
#1번째글은 15번째(0.00913391), 2번째글은 23번째(0.15965101), 3번째글은 16번째(0.13495328) 값인 것 확인!

H[:,:3]

array([[0.00112982, 0.        , 0.        ],
       [0.        , 0.00275125, 0.        ],
       [0.00473307, 0.        , 0.        ],
       [0.00561025, 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.00692328, 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.00091081, 0.        , 0.        ],
       [0.00448554, 0.        , 0.0008056 ],
       [0.0081309 , 0.00045932, 0.        ],
       [0.0079448 , 0.        , 0.        ],
       [0.00023681, 0.00809114, 0.        ],
       [0.00379817, 0.00038205, 0.        ],
       [0.00016232, 0.        , 0.        ],
       [0.00085665, 0.        , 0.        ],
       [0.00913391, 0.        , 0.        ],
       [0.00705321, 0.        , 0.13495328],
       [0.00240655, 0.00166718, 0.00135326],
       [0.        , 0.00551965, 0.        ],
       [0.0033583 , 0.00053603, 0.        ],
       [0.00361965, 0.00318245, 0.        ],
       [0.00617247, 0.00792015, 0.00478894],
       [0.

In [60]:
# 원래데이터('data')에서 'keywords'열만 뽑아서 'target'에 저장
target = list(data["keyword"])
print(target)

['디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로

In [65]:
# 'target'값(문자열)을 순서대로 '0 ~ 24'와 같은 숫자형으로 변환

num_target = []

for i in target:
    temp = i.replace('디지털 드로잉','0').replace('드로잉','1').replace('공예','2').replace('요리','3').replace('음료','4').replace('베이킹','5').replace('디저트','6').replace('음악','7').replace('운동','8').replace('라이프','9').replace('사진','10').replace('영상','11').replace('디자인','12').replace('개발','13').replace('데이터','14').replace('직무교육','15').replace('글쓰기','16').replace('언어','17').replace('외국어','18').replace('금융','19').replace('재테크','20').replace('창업','21').replace('부업','22').replace('성공 마인드','23').replace('아동 교육','24')
    num_target.append(int(temp))
    
print(num_target)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [82]:
len(num_target)

18554

In [66]:
from sklearn.metrics import confusion_matrix

#가로: 예측 클러스터 / 세로: 실제 클래스
pd.DataFrame(confusion_matrix(num_target, pred_labels))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0,1,0,0,0,0,0,3,0,0,...,1,1,0,0,1,0,1,1,2,0
1,18,7,2,1,62,3,10,14,3,7,...,5,3,0,0,8,2,22,12,8,1
2,47,8,7,5,16,14,13,19,4,9,...,4,8,1,1,6,3,28,23,12,0
3,158,10,6,7,28,41,25,19,3,14,...,12,32,4,5,23,8,342,55,52,1
4,140,9,11,13,80,54,15,19,8,38,...,14,104,6,10,47,15,67,87,83,9
5,67,5,8,4,17,29,10,25,5,11,...,13,18,0,4,17,0,281,20,31,1
6,99,10,10,18,27,19,24,18,7,12,...,10,29,0,6,21,16,352,107,56,0
7,198,7,14,9,57,52,38,18,8,46,...,4,40,3,5,58,6,23,208,44,10
8,84,4,7,5,34,613,17,13,10,15,...,13,14,4,3,20,3,19,19,17,6
9,240,15,48,36,68,54,42,31,13,36,...,13,38,3,21,33,32,52,44,51,4


In [73]:
cmatrix = confusion_matrix(num_target, pred_labels)

#10번째열 까지만 출력
cmatrix[:,:10]

array([[  0,   1,   0,   0,   0,   0,   0,   3,   0,   0],
       [ 18,   7,   2,   1,  62,   3,  10,  14,   3,   7],
       [ 47,   8,   7,   5,  16,  14,  13,  19,   4,   9],
       [158,  10,   6,   7,  28,  41,  25,  19,   3,  14],
       [140,   9,  11,  13,  80,  54,  15,  19,   8,  38],
       [ 67,   5,   8,   4,  17,  29,  10,  25,   5,  11],
       [ 99,  10,  10,  18,  27,  19,  24,  18,   7,  12],
       [198,   7,  14,   9,  57,  52,  38,  18,   8,  46],
       [ 84,   4,   7,   5,  34, 613,  17,  13,  10,  15],
       [240,  15,  48,  36,  68,  54,  42,  31,  13,  36],
       [201,  10,   9,   2,  28,  58,  16,  20,   4,  13],
       [209,  28,  22,  23, 104,  46,  49,  33,  14,  33],
       [156,  12,  43,  25, 123,  20,  18,  26,   5,  26],
       [109,   6,  53,  23,  53,  17,  23,  23,  44,  36],
       [112,   7,  32,  23,  68,   7,  21,  27,  46,  47],
       [ 38,   0,  35,   1,  18,   0,   2,  17,  12,  14],
       [ 68,  90,  25,   4,  37,   6,  15,  13,  10,  11

In [78]:
# 25개
labels_25 = list(cmatrix.argmax(axis=0))
print(labels_25)

[9, 16, 20, 20, 12, 8, 11, 21, 18, 22, 16, 21, 10, 4, 21, 20, 4, 14, 19, 7, 13, 6, 7, 22, 17]


In [80]:
#labels_25로 매칭했을 때, 전체 클러스터링 정확도 계산 과정(성능확인 위해서)

i = 0
sum_result = 0
for c in cmatrix.transpose():
    print(c)
    print('label:', labels_25[i])
    print('value:', c[labels_25[i]])
    i += 1
    sum_result += c[labels_25[i]]
    

[  0  18  47 158 140  67  99 198  84 240 201 209 156 109 112  38  68 164
  79 102  70 148 107  24  11]
label: 9
value: 240
[ 1  7  8 10  9  5 10  7  4 15 10 28 12  6  7  0 90 15 10  5 15 13  3  0
  1]
label: 16
value: 90
[ 0  2  7  6 11  8 10 14  7 48  9 22 43 53 32 35 25 14 33 53 68 46 17 11
 10]
label: 20
value: 68
[  0   1   5   7  13   4  18   9   5  36   2  23  25  23  23   1   4   3
   6  59 444  36 109  26   1]
label: 20
value: 444
[  0  62  16  28  80  17  27  57  34  68  28 104 123  53  68  18  37  29
  23  24  22 121  60   5   3]
label: 12
value: 123
[  0   3  14  41  54  29  19  52 613  54  58  46  20  17   7   0   6   9
  30  16   4  10  20   3   0]
label: 8
value: 613
[ 0 10 13 25 15 10 24 38 17 42 16 49 18 23 21  2 15 31 31  8 15 49 21  9
  0]
label: 11
value: 49
[ 3 14 19 19 19 25 18 18 13 31 20 33 26 23 27 17 13 32 28 20 22 54 41  7
  3]
label: 21
value: 54
[  0   3   4   3   8   5   7   8  10  13   4  14   5  44  46  12  10  79
 193  31  67  16  20   4   0]
label: 18
v

IndexError: list index out of range

In [81]:
# 맞은데이터 개수

print(sum_result)

646


In [83]:
# 맞은 비율

sum_result/len(num_target)

0.034817290072221625